In [108]:
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
import plotly.express as px
import plotly.graph_objects as go

In [78]:
df_bsc = pd.read_csv('bsc5.csv')
df_bsc = df_bsc.dropna(subset=['Parallax'])
df_bsc = df_bsc.reset_index(drop=True)

df_bsc['Distance'] = (1/df_bsc['Parallax'])
print(df_bsc['Distance'].head())

0    71.428571
1    21.276596
2    20.000000
3    14.925373
4    31.250000
Name: Distance, dtype: float64


In [79]:
df_bsc['RA_deg'] = np.nan
df_bsc['DE_deg'] = np.nan
for i in range(len(df_bsc)):
    ra_h = df_bsc.at[i, "RAh"]
    ra_m = df_bsc.at[i, "RAm"]
    ra_s = df_bsc.at[i, "RAs"]

    if pd.notnull(ra_h) and pd.notnull(ra_m) and pd.notnull(ra_s):
        RaDeg=((int(ra_h) / 24) * 360 +
              (int(ra_m) / 60) * 15 +
              (int(ra_s) / 3600) * 15)
        df_bsc.at[i, "RA_deg"] = RaDeg
            
    dec_sign = df_bsc.at[i, "DE-"]
    dec_deg = df_bsc.at[i, "DEd"]
    dec_min = df_bsc.at[i, "DEm"]
    dec_sec = df_bsc.at[i, "DEs"]
    
    if pd.notnull(dec_sign) and pd.notnull(dec_deg) and pd.notnull(dec_min) and pd.notnull(dec_sec):
        declination = dec_deg + (dec_min / 60) + (dec_sec / 3600)
        if dec_sign == '-':
            declination = -declination
        df_bsc.at[i, "DE_deg"] = declination

coords = SkyCoord(df_bsc['RA_deg'], df_bsc['DE_deg'], unit=(u.deg, u.deg))
df_bsc['x'] = coords.cartesian.x.value * df_bsc['Distance']
df_bsc['y'] = coords.cartesian.y.value * df_bsc['Distance']
df_bsc['z'] = coords.cartesian.z.value * df_bsc['Distance']

In [112]:
spectral_colors = {
    'O': 'darkblue',
    'B': 'dodgerblue',
    'A': 'deepskyblue',
    'F': 'green',
    'G': 'gold',
    'K': 'orange',
    'M': 'red'
}
df_bsc['Color'] = np.nan
for i in range(len(df_bsc)):
    spec_type=df_bsc.at[i, "SpType"]
    state=True
    for ctype in spectral_colors:
        if spec_type[0]==ctype:
            df_bsc.at[i, "Color"] = spectral_colors[ctype]
            state=False
            break
    if state:
        df_bsc.at[i, "Color"] = 'black'
print(df_bsc['Color'].head())

0        orange
1          gold
2          gold
3        orange
4    dodgerblue
Name: Color, dtype: object


In [113]:
fig = go.Figure(data=go.Scatter3d(
    x=df_bsc['x'],
    y=df_bsc['y'],
    z=df_bsc['z'],
    mode='markers',
    marker=dict(
        size=5,
        color=df_bsc['Color'].astype('category').cat.codes,
        opacity=0.8
    ),
    hovertemplate='<b>HR:</b> %{text[0]}<br>'
                  '<b>Distance (parsec):</b> %{text[1]:.2f}<br>'
                  '<b>Radial Velocity (km/s):</b> %{text[2]:.2f}',
    text=df_bsc[['HR', 'Distance', 'RadVel']],
))

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

fig.show()